Introduction
In this notebook , we are going to Generate Art using GANS. Explaining it in a brief way , it is a Artist who forges Ukiyoe paintings , but using random normal picture that is assigned to him. YES you can have your own portrait made in Ukiyoe style!!! However , this Artist has both his ears

In [ ]:
# %tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# try:
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
#     print('Device:', tpu.master())
#     tf.config.experimental_connect_to_cluster(tpu)
#     tf.tpu.experimental.initialize_tpu_system(tpu)
#     strategy = tf.distribute.experimental.TPUStrategy(tpu)
# except:
#     strategy = tf.distribute.get_strategy()
# print('Number of replicas:', strategy.num_replicas_in_sync)

In [ ]:
# pip install git+https://www.github.com/keras-team/keras-contrib.git

In [ ]:
import tensorflow_datasets as tfds
 
import os
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Concatenate
from tensorflow import keras
#from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from tensorflow.keras.models import Model

AUTOTUNE = tf.data.AUTOTUNE

Loading the Data:
We are using the cezanne2photo data from TensorFlow Datasets , from the cycle_gan category. However we are inverting the data , that is photo to cezanne.

In [ ]:
dataset, metadata = tfds.load('cycle_gan/cezanne2photo',
                              with_info=True, as_supervised=True,try_gcs=True)

train_normal, train_van = dataset['trainB'], dataset['trainA']
test_normal, test_van = dataset['testB'], dataset['testA']

In [ ]:
BUFFER_SIZE = 1000
BATCH_SIZE = 1
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_SHAPE = (256,256,3)

In [ ]:
def random_crop(image):
  cropped_image = tf.image.random_crop(
      image, size=[IMG_HEIGHT, IMG_WIDTH, 3])

  return cropped_image

In [ ]:
def normalize(image):
  image = tf.cast(image, tf.float32)
  image = (image / 127.5) -1
  return image

In [ ]:
def random_jitter(image):
  # resizing to 286 x 286 x 3
  image = tf.image.resize(image, [286, 286],
                          method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

  # randomly cropping to 256 x 256 x 3
  image = random_crop(image)

  # random mirroring
  image = tf.image.random_flip_left_right(image)

  return image

In [ ]:
def preprocess_image_train(image, label):
  image = random_jitter(image)
  image = normalize(image)
  return image

In [ ]:
def preprocess_image_test(image, label):
  image = random_jitter(image)
  image = normalize(image)
  return image

In [ ]:
train_normal = train_normal.map(
    preprocess_image_train, num_parallel_calls=AUTOTUNE).shuffle(
    BUFFER_SIZE).batch(1)

train_van = train_van.map(
    preprocess_image_train, num_parallel_calls=AUTOTUNE).shuffle(
    BUFFER_SIZE).batch(1)

test_normal = test_normal.map(
    preprocess_image_test, num_parallel_calls=AUTOTUNE).shuffle(
    BUFFER_SIZE).batch(1)

test_van = test_van.map(
    preprocess_image_test, num_parallel_calls=AUTOTUNE).shuffle(
    BUFFER_SIZE).batch(1)

In [ ]:
sample_normal = next(iter(train_normal))
sample_van = next(iter(train_van))

In [ ]:
plt.subplot(121)
plt.title('Normal pic')
plt.imshow(sample_normal[0] * 0.5 + 0.5)

plt.subplot(122)
plt.title('Normal pic with random jitter')
plt.imshow(random_jitter(sample_normal[0]) * 0.5 + 0.5)

In [ ]:
plt.subplot(121)
plt.title('cezanne Painting')
plt.imshow(sample_van[0] * 0.5 + 0.5)

plt.subplot(122)
plt.title('cezanne Painting with random jitter')
plt.imshow(random_jitter(sample_van[0]) * 0.5 + 0.5)

Setting the Models:
We are implementing CycleGan , with a modified UNET model as a generator , and a PatchGan block as a Discriminator. The difference between a normal block and a GAN block is that it uses a InstanceNormalization..

In [ ]:
OUTPUT_CHANNELS = 3

In [ ]:
def downsample(filters, size, norm_type='batchnorm', apply_norm=True):
  """Downsamples an input.
  Conv2D => Batchnorm => LeakyRelu
  Args:
    filters: number of filters
    size: filter size
    norm_type: Normalization type; either 'batchnorm' or 'instancenorm'.
    apply_norm: If True, adds the batchnorm layer
  Returns:
    Downsample Sequential Model
  """
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
      tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))

  if apply_norm:
    if norm_type.lower() == 'batchnorm':
      result.add(tf.keras.layers.BatchNormalization())
    elif norm_type.lower() == 'instancenorm':
      result.add(InstanceNormalization())

  result.add(tf.keras.layers.LeakyReLU())

  return result

In [ ]:
down_model = downsample(3, 4)
down_result = down_model(tf.expand_dims(sample_van[0], 0))
print (down_result.shape)

In [ ]:
def upsample(filters, size, norm_type='batchnorm', apply_dropout=False):
  """Upsamples an input.
  Conv2DTranspose => Batchnorm => Dropout => Relu
  Args:
    filters: number of filters
    size: filter size
    norm_type: Normalization type; either 'batchnorm' or 'instancenorm'.
    apply_dropout: If True, adds the dropout layer
  Returns:
    Upsample Sequential Model
  """

  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
      tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                      padding='same',
                                      kernel_initializer=initializer,
                                      use_bias=False))

  if norm_type.lower() == 'batchnorm':
    result.add(tf.keras.layers.BatchNormalization())
  elif norm_type.lower() == 'instancenorm':
    result.add(InstanceNormalization())

  if apply_dropout:
    result.add(tf.keras.layers.Dropout(0.5))

  result.add(tf.keras.layers.ReLU())

  return result

In [ ]:
up_model = upsample(3, 4)
up_result = up_model(down_result)
print (up_result.shape)

The Generator :
There are 2 generators (A and B) and 2 discriminators (X and Y) being trained here.

Generator G learns to transform image X to image Y. (A:X−>Y) Generator F learns to transform image Y to image X. (B:Y−>X) Discriminator D_X learns to differentiate between image X and generated image X (B(Y)). Discriminator D_Y learns to differentiate between image Y and generated image Y (A(X)).

In [ ]:
def resnet_block(n_filters, input_layer):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# first layer convolutional layer
	g = Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(input_layer)
	g = tf.keras.layers.BatchNormalization()(g)
	g = Activation('relu')(g)
	# second convolutional layer
	g = Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(g)
	g = tf.keras.layers.BatchNormalization()(g)
	# concatenate merge channel-wise with input layer
	g = Concatenate()([g, input_layer])
	return g

In [ ]:
def define_generator(image_shape, n_resnet=4):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# image input
	in_image = Input(shape=image_shape)
	# c7s1-64
	g = Conv2D(32, (8,8), padding='same', kernel_initializer=init)(in_image)
	g =tf.keras.layers.BatchNormalization()(g)
	g = Activation('relu')(g)
	# d128
	g = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = tf.keras.layers.BatchNormalization()(g)
	g = Activation('relu')(g)
	# d256
	g = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g =tf.keras.layers.BatchNormalization()(g)
	g = Activation('relu')(g)
 
 # d512
# 	g = Conv2D(256, (2,2), strides=(2,2), padding='same', kernel_initializer=init)(g)
# 	g =tf.keras.layers.BatchNormalization()(g)
# 	g = Activation('relu')(g)
	# R512
	for _ in range(n_resnet):
		g = resnet_block(128, g)
	# 	# u256
	# 	g = Conv2DTranspose(256, (5,5), strides=(2,2), padding='same', kernel_initializer=init)(g)
	# g = tf.keras.layers.BatchNormalization()(g)
	# g = Activation('relu')(g)
	# u128
	g = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = tf.keras.layers.BatchNormalization()(g)
	g = Activation('relu')(g)
	# u64
	g = Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = tf.keras.layers.BatchNormalization()(g)
	g = Activation('relu')(g)
 	# u64
# 	g = Conv2DTranspose(32, (2,2), strides=(2,2), padding='same', kernel_initializer=init)(g)
# 	g = tf.keras.layers.BatchNormalization()(g)
# 	g = Activation('relu')(g)
	# c7s1-3
	g = Conv2D(3, (8,8), padding='same', kernel_initializer=init)(g)
	g = tf.keras.layers.BatchNormalization()(g)
	out_image = Activation('tanh')(g)
	# define model
	model = Model(in_image, out_image)
	return model

In [ ]:
generator = define_generator(IMG_SHAPE)
tf.keras.utils.plot_model(generator, show_shapes=True, dpi=64)
generator.summary()

In [ ]:


def discriminator(norm_type='batchnorm', target=True):
  """PatchGan discriminator model (https://arxiv.org/abs/1611.07004).
  Args:
    norm_type: Type of normalization. Either 'batchnorm' or 'instancenorm'.
    target: Bool, indicating whether target image is an input or not.
  Returns:
    Discriminator model
  """

  initializer = tf.random_normal_initializer(0., 0.02)

  inp = tf.keras.layers.Input(shape=[None, None, 3], name='input_image')
  x = inp

  if target:
    tar = tf.keras.layers.Input(shape=[None, None, 3], name='target_image')
    x = tf.keras.layers.concatenate([inp, tar])  # (bs, 256, 256, channels*2)

  down1 = downsample(64, 4, norm_type, False)(x)  # (bs, 128, 128, 64)
  down2 = downsample(128, 4, norm_type)(down1)  # (bs, 64, 64, 128)
  down3 = downsample(256, 4, norm_type)(down2)  # (bs, 32, 32, 256)

  zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3)  # (bs, 34, 34, 256)
  conv = tf.keras.layers.Conv2D(
      512, 4, strides=1, kernel_initializer=initializer,
      use_bias=False)(zero_pad1)  # (bs, 31, 31, 512)

  if norm_type.lower() == 'batchnorm':
    norm1 = tf.keras.layers.BatchNormalization()(conv)
  elif norm_type.lower() == 'instancenorm':
    norm1 = InstanceNormalization(axis=-1)(conv)

  leaky_relu = tf.keras.layers.LeakyReLU()(norm1)

  zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu)  # (bs, 33, 33, 512)

  last = tf.keras.layers.Conv2D(
      1, 4, strides=1,
      kernel_initializer=initializer)(zero_pad2)  # (bs, 30, 30, 1)

  if target:
    return tf.keras.Model(inputs=[inp, tar], outputs=last)
  else:
    return tf.keras.Model(inputs=inp, outputs=last)


In [ ]:
generator_a = define_generator(IMG_SHAPE)
generator_b = define_generator(IMG_SHAPE)

discriminator_x = discriminator(norm_type='batchnorm', target=False)
discriminator_y = discriminator(norm_type='batchnorm', target=False)

In [ ]:
to_van = generator_a(sample_normal)
to_normal = generator_b(sample_van)
plt.figure(figsize=(8, 8))
contrast = 8

imgs = [sample_normal, to_van, sample_van, to_normal]
title = ['Normal', 'To Ukiyoe', 'Ukiyoe', 'To Normal']

for i in range(len(imgs)):
  plt.subplot(2, 2, i+1)
  plt.title(title[i])
  if i % 2 == 0:
    plt.imshow(imgs[i][0] * 0.5 + 0.5)
  else:
    plt.imshow(imgs[i][0] * 0.5 * contrast + 0.5)
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))

plt.subplot(121)
plt.title('Is a real Ukiyoe Painting?')
plt.imshow(discriminator_y(sample_van)[0, ..., -1], cmap='RdBu_r')

plt.subplot(122)
plt.title('Is a real Normal Pic?')
plt.imshow(discriminator_x(sample_normal)[0, ..., -1], cmap='RdBu_r')

plt.show()

In [ ]:
LAMBDA = 8

In [ ]:
loss_obj = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(real, generated):
  real_loss = loss_obj(tf.ones_like(real), real)

  generated_loss = loss_obj(tf.zeros_like(generated), generated)

  total_disc_loss = real_loss + generated_loss

  return total_disc_loss * 0.5

In [ ]:
def generator_loss(generated):
  return loss_obj(tf.ones_like(generated), generated)

In [ ]:
def calc_cycle_loss(real_image, cycled_image):
  loss1 = tf.reduce_mean(tf.abs(real_image - cycled_image))
  
  return LAMBDA * loss1

In [ ]:
def identity_loss(real_image, same_image):
  loss = tf.reduce_mean(tf.abs(real_image - same_image))
  return LAMBDA * 0.5 * loss

In [ ]:
generator_a_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.7)
generator_b_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.7)
discriminator_x_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)
discriminator_y_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)

In [ ]:
# checkpoint_path = "./checkpoints/train"

# ckpt = tf.train.Checkpoint(generator_a=generator_a,
#                            generator_b=generator_b,
#                            discriminator_x=discriminator_x,
#                            discriminator_y=discriminator_y,
#                            generator_a_optimizer=generator_a_optimizer,
#                            generator_b_optimizer=generator_b_optimizer,
#                            discriminator_x_optimizer=discriminator_x_optimizer,
#                            discriminator_y_optimizer=discriminator_y_optimizer)

# ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# # if a checkpoint exists, restore the latest checkpoint.
# if ckpt_manager.latest_checkpoint:
#   ckpt.restore(ckpt_manager.latest_checkpoint)
#   print ('Latest checkpoint restored!!')

In [ ]:
EPOCHS = 200

In [ ]:
def generate_images(model, test_input):
  prediction = model(test_input)      
  plt.figure(figsize=(12, 12))
 
  display_list = [test_input[0], prediction[0]]
  title = ['Input Image', 'Predicted Painting']
 
 
  for i in range(2):
    plt.subplot(1, 2, i+1)
    plt.title(title[i])
    # getting the pixel values between [0, 1] to plot it.
    plt.imshow(display_list[i] * 0.5 + 0.5)
    plt.axis('off')
  plt.show()
  return prediction

In [ ]:
@tf.function
def train_step(real_x, real_y):
  # persistent is set to True because the tape is used more than
  # once to calculate the gradients.
  with tf.GradientTape(persistent=True) as tape:
    # Generator G translates X -> Y
    # Generator F translates Y -> X.
    
    fake_y = generator_a(real_x, training=True)
    cycled_x = generator_b(fake_y, training=True)

    fake_x = generator_b(real_y, training=True)
    cycled_y = generator_a(fake_x, training=True)

    # same_x and same_y are used for identity loss.
    same_x = generator_b(real_x, training=True)
    same_y = generator_a(real_y, training=True)

    disc_real_x = discriminator_x(real_x, training=True)
    disc_real_y = discriminator_y(real_y, training=True)

    disc_fake_x = discriminator_x(fake_x, training=True)
    disc_fake_y = discriminator_y(fake_y, training=True)

    # calculate the loss
    gen_a_loss = generator_loss(disc_fake_y)
    gen_b_loss = generator_loss(disc_fake_x)
    
    total_cycle_loss = calc_cycle_loss(real_x, cycled_x) + calc_cycle_loss(real_y, cycled_y)
    
    # Total generator loss = adversarial loss + cycle loss
    total_gen_a_loss = gen_a_loss + total_cycle_loss + identity_loss(real_y, same_y)
    total_gen_b_loss = gen_b_loss + total_cycle_loss + identity_loss(real_x, same_x)

    disc_x_loss = discriminator_loss(disc_real_x, disc_fake_x)
    disc_y_loss = discriminator_loss(disc_real_y, disc_fake_y)
  
  # Calculate the gradients for generator and discriminator
  generator_a_gradients = tape.gradient(total_gen_a_loss, 
                                        generator_a.trainable_variables)
  generator_b_gradients = tape.gradient(total_gen_b_loss, 
                                        generator_b.trainable_variables)
  
  discriminator_x_gradients = tape.gradient(disc_x_loss, 
                                            discriminator_x.trainable_variables)
  discriminator_y_gradients = tape.gradient(disc_y_loss, 
                                            discriminator_y.trainable_variables)
  
  # Apply the gradients to the optimizer
  generator_a_optimizer.apply_gradients(zip(generator_a_gradients, 
                                            generator_a.trainable_variables))

  generator_b_optimizer.apply_gradients(zip(generator_b_gradients, 
                                            generator_b.trainable_variables))
  
  discriminator_x_optimizer.apply_gradients(zip(discriminator_x_gradients,
                                                discriminator_x.trainable_variables))
  
  discriminator_y_optimizer.apply_gradients(zip(discriminator_y_gradients,
                                                discriminator_y.trainable_variables))

In [ ]:
for epoch in range(EPOCHS):
  start = time.time()

  n = 0
  for image_x, image_y in tf.data.Dataset.zip((train_normal, train_van)):
    train_step(image_x, image_y)
    if n % 10 == 0:
      print ('.', end='')
    n+=1

  clear_output(wait=True)
  # Using a consistent image (sample_horse) so that the progress of the model
  # is clearly visible.
  generate_images(generator_a, sample_normal)

#   if (epoch + 1) % 5 == 0:
#     ckpt_save_path = ckpt_manager.save()
#     print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
#                                                          ckpt_save_path))

  print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                      time.time()-start))

In [ ]:
for inp in test_normal.take(10):
  generate_images(generator_a, inp)

In [ ]:

generator_Save=generator_a.save('./cezanne.h5')

In [ ]:
model_load = keras.models.load_model('./cezanne.h5')

In [4]:
imgurl="https://upload.wikimedia.org/wikipedia/commons/thumb/e/ec/Mona_Lisa%2C_by_Leonardo_da_Vinci%2C_from_C2RMF_retouched.jpg/687px-Mona_Lisa%2C_by_Leonardo_da_Vinci%2C_from_C2RMF_retouched.jpg"

In [5]:
from urllib.request import urlopen
from PIL import Image
import cv2
from numpy import asarray
from scipy import misc

img = Image.open(urlopen(imgurl))
basewidth = 256
#wpercent = (basewidth / float(img.size[0]))
hsize = 256#int((float(img.size[1]) * float(wpercent)))
img = img.resize((basewidth, hsize))
#img.save('resized_image.jpg')
#img.show()
#img = cv2.cvtColor(np.float32(img), cv2.COLOR_RGB2BGR)
#image=misc.imread(img)
img=asarray(img)
img.astype('float32')
img=img/255.0
image_tensor = tf.convert_to_tensor(img, dtype=tf.float64)
image_tensor = tf.expand_dims(image_tensor, 0)
generate_images(model_load, image_tensor)